In [ ]:
import pandas as pd
import IGtools as igt
import networkx as nx
import os, sys 
import matplotlib.pyplot as plt
import pandas as pd
from mis_utils import *

In [ ]:
import igraph as ig

In [ ]:
sizes = [20,40,50,60,70,80,90,100]

In [ ]:
zs = [3,5,8,15]

g_test = {}
deg_test = {}
lg_test = {}
deg_lg_test = {}
ig_test = {}
deg_ig_test = {}
corresponding_degree = {}
boosted_degree_corrected = {}

fig = plt.figure(figsize=(12,8))
for z in zs:
    g_test[z] = {}
    g_test[z] = {}
    deg_test[z] = {}
    lg_test[z] = {}
    deg_lg_test[z] = {}
    ig_test[z] = {}
    deg_ig_test[z] = {}
    boosted_degree_corrected[z] = {}
    corresponding_degree[z] = {}
    for j, size in enumerate(sizes):
        g_test[z][j] = igt.create_ER_bipartite_graph(size,z/float(size-1))
        g_test[z][j] = nx.Graph(g_test[z][j])
        deg_test[z][j] = dict(nx.degree(g_test[z][j]));
        lg_test[z][j] = igt.line_graph(g_test[z][j]);
        deg_lg_test[z][j] = dict(nx.degree(lg_test[z][j]))
        ig_test[z][j] = igt.interference_graph(lg_test[z][j]);
        deg_ig_test[z][j] = dict(nx.degree(ig_test[z][j]))

        tt = [g_test[z][j].degree()[i] for i in range(size)]
        ss = [g_test[z][j].degree()[i] for i in range(size,2*size)]

        boosted_degree_corrected[z][j] = []
        corresponding_degree[z][j] = []
        for edge in g_test[z][j].edges():
            boosted_degree_corrected[z][j].append(igt.wedge_closure_boosted_degree_corrected(deg_test[z][j][edge[0]], deg_test[z][j][edge[1]], np.array(ss), np.array(tt)));

            corresponding_degree[z][j].append(deg_ig_test[z][j][str(list(edge))])
        plt.subplot(4,2,j+1)
        plt.plot(corresponding_degree[z][j], boosted_degree_corrected[z][j],'o',label=z)
        plt.title(str(size))
        plt.plot(range(np.max(corresponding_degree[z][j])),range(np.max(corresponding_degree[z][j])),'k--')

    plt.tight_layout()

In [ ]:
import seaborn as sns 
plt.style.use('seaborn-white')

In [ ]:
# define the colormap
cmap = plt.cm.hot
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

In [ ]:
fs = 30
viz = [0,7]
size_vis = [20,100]
max_lim = [250,600]
colors = ['b', 'r', 'y', 'g','b']
plt.style.use('seaborn-white')
fig = plt.figure(figsize=(10,8))
for jj,j in enumerate(viz):
    for zz,z in enumerate(zs):
        plt.subplot(2,2,jj+1)
        plt.scatter(np.array(corresponding_degree[z][j]), np.array(boosted_degree_corrected[z][j]),
                 s=40,c=cmap(zz*256/len(zs)),label='z='+str(z),alpha=.8)

        plt.title('BG layer size = '+str(size_vis[jj]))
        plt.ylabel(r'$\tilde{k}^{IG}$', fontsize=fs)        
        plt.legend(loc=0, fontsize=20)
        plt.xlim(0,max_lim[jj])
        plt.xticks(fontsize=fs-5)
        plt.yticks(fontsize=fs-5)        


        plt.box('off')
        plt.plot(np.array(range(np.max(corresponding_degree[z][j]))),
                 np.array(range(np.max(corresponding_degree[z][j]))),'k--',alpha=0.6)
        ax = plt.subplot(2,2,jj+3)
        sns.distplot(corresponding_degree[z][j],ax=ax,kde=False,label = r'$k^{IG}$', norm_hist=True, color=cmap(zz*256/len(zs)))
        sns.distplot(boosted_degree_corrected[z][j],hist=False,kde=True,ax=ax,color=cmap(zz*256/len(zs)))
        plt.grid('off')
        plt.box('off')
        plt.xlabel(r'$k^{IG}$', fontsize=fs)
        plt.ylabel(r'$p(k^{IG})$', fontsize=fs)
        plt.xticks(fontsize=fs-5)
        plt.yticks(fontsize=fs-5)        
        plt.xlim(0,max_lim[jj])
        plt.legend(loc=0, fontsize=20)
plt.tight_layout()
    

In [ ]:
from scipy.stats import pearsonr
pearsonr(np.array(corresponding_degree[z][j]), np.array(boosted_degree_corrected[z][j]))

In [ ]:
import pickle as pk
fig2a = {'corresponding_degree': corresponding_degree, 'predicted_degree': boosted_degree_corrected}


pk.dump(fig2a, open('./figure_data/figure2a.pck', 'wb'))